# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description. 

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |


In [26]:
# Write your answer to Task 1 here

import pandas as pd
import numpy as np

# Load data
production_data = pd.read_csv('production_data.csv')

# Cleaning Process

# 1. Convert production_date to datetime
production_data['production_date'] = pd.to_datetime(production_data['production_date'], errors='coerce')

# 2. Clean and map raw_material_supplier
production_data['raw_material_supplier'] = production_data['raw_material_supplier'].astype(str).str.strip().str.lower()
production_data['raw_material_supplier'] = production_data['raw_material_supplier'].map({
    '1': 'national_supplier', '2': 'international_supplier'
})
production_data['raw_material_supplier'].fillna('national_supplier', inplace=True)

# 3. Clean and validate pigment_type
production_data['pigment_type'] = production_data['pigment_type'].astype(str).str.strip().str.lower()
valid_pigments = ['type_a', 'type_b', 'type_c']
production_data['pigment_type'] = production_data['pigment_type'].where(
    production_data['pigment_type'].isin(valid_pigments), 'other'
)

# 4. pigment_quantity - Range 1-100, replace invalid or missing with median
median_pigment_quantity = production_data['pigment_quantity'].where(
    (production_data['pigment_quantity'] >= 1) & (production_data['pigment_quantity'] <= 100)
).median()
production_data['pigment_quantity'] = production_data['pigment_quantity'].where(
    (production_data['pigment_quantity'] >= 1) & (production_data['pigment_quantity'] <= 100),
    median_pigment_quantity
)

# 5. mixing_time - replace missing with mean, rounded to 2 decimals
mean_mixing_time = round(production_data['mixing_time'].mean(), 2)
production_data['mixing_time'].fillna(mean_mixing_time, inplace=True)
production_data['mixing_time'] = production_data['mixing_time'].round(2)

# 6. Clean and validate mixing_speed
production_data['mixing_speed'] = production_data['mixing_speed'].astype(str).str.strip().str.title()
valid_speeds = ['Low', 'Medium', 'High']
production_data['mixing_speed'] = production_data['mixing_speed'].where(
    production_data['mixing_speed'].isin(valid_speeds), 'Not Specified'
)

# 7. product_quality_score - Should be 1 to 10, replace invalid or missing with mean, rounded to 2 decimals
mean_quality_score = round(production_data['product_quality_score'].where(
    (production_data['product_quality_score'] >= 1) & (production_data['product_quality_score'] <= 10)
).mean(), 2)
production_data['product_quality_score'] = production_data['product_quality_score'].where(
    (production_data['product_quality_score'] >= 1) & (production_data['product_quality_score'] <= 10),
    mean_quality_score
).round(2)

# Final clean DataFrame
clean_data = production_data.copy()

# Preview cleaned data
clean_data.head()


,batch_id,production_date,raw_material_supplier,pigment_type,pigment_quantity,mixing_time,mixing_speed,product_quality_score
0,1,2024-06-25,national_supplier,type_a,42.822881,22.56,High,7.17
1,2,2023-11-23,international_supplier,type_b,42.873479,44.97,High,6.85
2,3,2024-02-18,international_supplier,type_b,33.251206,52.62,High,5.66
3,4,2023-11-11,national_supplier,type_a,52.532817,24.96,High,6.99
4,5,2024-04-11,international_supplier,type_c,36.205108,25.37,High,7.10


# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [27]:
# Write your answer to Task 2 here
import pandas as pd
import numpy as np

production_data=pd.read_csv('production_data.csv')


# Clean raw_material_supplier as before
production_data['raw_material_supplier'] = production_data['raw_material_supplier'].map({1: 'national_supplier', 2: 'international_supplier'})
production_data['raw_material_supplier'].fillna('national_supplier', inplace=True)

# Group by supplier and calculate averages
aggregated_data = production_data.groupby('raw_material_supplier').agg(
    avg_product_quality_score=('product_quality_score', 'mean'),
    avg_pigment_quantity=('pigment_quantity', 'mean')
).reset_index()

# Round to 2 decimals
aggregated_data = aggregated_data.round(2)

aggregated_data


,raw_material_supplier,avg_product_quality_score,avg_pigment_quantity
0,international_supplier,5.97,34.91
1,national_supplier,8.02,44.73


# Task 3

To get more insight into the factors behind product quality, you want to filter the data to see an average product quality score for a specified set of results.

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named pigment_data.
- It should consist of a 1-row DataFrame with 3 columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places where appropriate.


In [28]:
# Write your answer to Task 3 here

import pandas as pd
import numpy as np

df=pd.read_csv('production_data.csv')

# Convert 'raw_material_supplier' to numeric (it may be read as string)
df["raw_material_supplier"] = pd.to_numeric(df["raw_material_supplier"], errors="coerce")

# Filter the data
filtered_df = df[
    (df["raw_material_supplier"] == 2) &
    (df["pigment_quantity"] > 35)
]

# Calculate the average product quality score
avg_score = round(filtered_df["product_quality_score"].mean(), 2)
avg_pigment_quantity = round(filtered_df["pigment_quantity"].mean(), 2)

# Create the resulting DataFrame
pigment_data = pd.DataFrame({
    "raw_material_supplier": [2],
    "pigment_quantity": [avg_pigment_quantity],
    "avg_product_quality_score": [avg_score]
})

pigment_data



,raw_material_supplier,pigment_quantity,avg_product_quality_score
0,2,39.01,5.97


# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [29]:
# Write your answer to Task 4 here


import pandas as pd
import numpy as np

df=pd.read_csv('production_data.csv')

# Calculate means and standard deviations
product_quality_score_mean = round(df['product_quality_score'].mean(), 2)
product_quality_score_sd = round(df['product_quality_score'].std(), 2)
pigment_quantity_mean = round(df['pigment_quantity'].mean(), 2)
pigment_quantity_sd = round(df['pigment_quantity'].std(), 2)

# Calculate Pearson correlation coefficient
corr_coef = round(df[['pigment_quantity', 'product_quality_score']].corr().iloc[0, 1], 2)

# Create the output DataFrame
product_quality = pd.DataFrame([{
    'product_quality_score_mean': product_quality_score_mean,
    'product_quality_score_sd': product_quality_score_sd,
    'pigment_quantity_mean': pigment_quantity_mean,
    'pigment_quantity_sd': pigment_quantity_sd,
    'corr_coef': corr_coef
}])

product_quality

,product_quality_score_mean,product_quality_score_sd,pigment_quantity_mean,pigment_quantity_sd,corr_coef
0,6.68,1.39,38.35,6.83,0.49


# FORMATTING AND NAMING CHECK
Use the code block below to check that your outputs are correctly named and formatted before you submit your project.

This code checks whether you have met our automarking requirements: that the specified DataFrames exist and contain the required columns. It then prints a table showing ✅ for each column that exists and ❌ for any that are missing, or if the DataFrame itself isn't available.

If a DataFrame or a column in a DataFrame doesn't exist, carefully check your code again.

IMPORTANT: even if your code passes the check below, this does not mean that your entire submission is correct. This is a check for naming and formatting only.

In [30]:
import pandas as pd

def check_columns(output_df, output_df_name, required_columns):
    results = []
    for col in required_columns:
        exists = col in output_df.columns
        results.append({'Dataset': output_df_name, 'Column': col, 'Exists': '✅' if exists else '❌'})
    return results

def safe_check(output_df_name, required_columns):
    results = []
    if output_df_name in globals():
        obj = globals()[output_df_name]
        if isinstance(obj, pd.DataFrame):
            results.extend(check_columns(obj, output_df_name, required_columns))
        elif isinstance(obj, str) and ("SELECT" in obj.upper() or "FROM" in obj.upper()):
            results.append({'Dataset': output_df_name, 'Column': '—', 'Exists': 'ℹ️ SQL query string'})
        else:
            results.append({'Dataset': output_df_name, 'Column': '—', 'Exists': '❌ Not a DataFrame or query'})
    else:
        results.append({'Dataset': output_df_name, 'Column': '—', 'Exists': '❌ Variable not defined'})
    return results

requirements = {
    'clean_data': ['production_date', 'pigment_type', 'mixing_time', 'mixing_speed'],
    'aggregated_data': ['raw_material_supplier', 'avg_product_quality_score', 'avg_pigment_quantity'],
    'pigment_data': ['raw_material_supplier', 'pigment_quantity', 'avg_product_quality_score'],
    'product_quality': ['product_quality_score_mean', 'product_quality_score_sd',
                        'pigment_quantity_mean', 'pigment_quantity_sd', 'corr_coef']
}

all_results = []
for output_df_name, cols in requirements.items():
    all_results += safe_check(output_df_name, cols)

check_results_df = pd.DataFrame(all_results)

print(check_results_df)

            Dataset                      Column Exists
0        clean_data             production_date      ✅
1        clean_data                pigment_type      ✅
2        clean_data                 mixing_time      ✅
3        clean_data                mixing_speed      ✅
4   aggregated_data       raw_material_supplier      ✅
5   aggregated_data   avg_product_quality_score      ✅
6   aggregated_data        avg_pigment_quantity      ✅
7      pigment_data       raw_material_supplier      ✅
8      pigment_data            pigment_quantity      ✅
9      pigment_data   avg_product_quality_score      ✅
10  product_quality  product_quality_score_mean      ✅
11  product_quality    product_quality_score_sd      ✅
12  product_quality       pigment_quantity_mean      ✅
13  product_quality         pigment_quantity_sd      ✅
14  product_quality                   corr_coef      ✅
